In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Familiarizing ourselves with the data

Let's run a couple of queries to get to know the data

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


For reference, here is a breakdown of the meanings of the column names for the facts table:


name - The name of the country.
area - The total land and sea area of the country.
population - The country's population.
population_growth- The country's population growth as a percentage.
birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
death_rate - The country's death rate, or the number of death a year per 1,000 people.
area- The country's total area (both land and water).
area_land - The country's land area in square kilometers.
area_water - The country's water area in square kilometers.

# Outliers
Next, we'll calculate some summary statistics to explore the existence of outliers in the data.

In [4]:
%%sql
SELECT MIN(population) AS 'minimum_population',
       MAX(population) AS 'maximum_population',
       MIN(population_growth) AS 'minimum_population_growth',
       MAX(population_growth) AS 'maximum_population_growth'
  FROM facts;

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,7256490011,0.0,4.02


## Observations of Outliers
It looks like we have minimum population and population growth figures of 0. We also have a maximum population of over 7.2 billion—that's more like the population of the entire world, 

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


and in-fact is—the recorded population for the entire world. Next, we'll explore the minimum population centers.

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Antarctica makes sense; there should be no recorded population in Antarctica.

Next, we'll calculate our summary statistics and exclude the outlying row with the name `World` from our query, and then calculate the averages of the `population` and `area` fields in order to start exploring population density.

In [7]:
%%sql
SELECT MIN(population) AS 'min_population',
       MAX(population) AS 'max_population',
       MIN(population_growth) AS 'min_population_growth',
       MAX(population_growth) AS 'max_population_growth'
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


There is a country with a population of ~1.4 billion people, a significant portion of the world population.

# Population Density

When exploring average population density, we want to remember to exclude our `World` entry in the table, since it will skew our average calculation to be too high.

In [8]:
%%sql
SELECT AVG(population) AS 'average_population', AVG(area) AS 'average_area'
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


average_population,average_area
32242666.56846473,555093.546184739


We can use versions of this query as a subquery with the `WHERE` keyword in order to calculate and filter our query by population density; what we are doing is selecting countries in which the population is above average and the area is below average.

In [9]:
%%sql
SELECT *
  FROM facts
 WHERE (population > (SELECT AVG(population)
                        FROM facts
                       WHERE name <> 'World')
       AND area < (SELECT AVG(area)
                     FROM facts
                    WHERE name <> 'World')
       );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


A lot of these names ring bells as "densely populated", so there is a good chance that we did everything we intended to and have a list of densely populated countries.

# Further Questions
* Which countries have the highest population/area ratio

# Country with the most people

In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                       WHERE name <> 'World'
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


according to the [US census Bureau](https://www.census.gov/popclock/print.php?component=counter), China has the highest population in the world of any country at the time of writing this Jupyter Notebook cell (2021), so this checks out with our results.

This query returns the country with the highest population, but what is the country with the highest population growth?

# Country with the highest population growth

In [11]:
%%sql
SELECT *
  FROM facts
 WHERE population_growth == (SELECT MAX(population_growth)
                               FROM facts
                              WHERE name <> 'World'
                            );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan, at the time this data was published, has a growth rate of over 400% its population!

# Countries with the highest water-to-land ratios

In [12]:
%%sql
SELECT *
  FROM facts
 WHERE (area_water/area_land) >= (SELECT AVG(area_water)/AVG(area_land)
                                    FROM facts
                                   WHERE name <> 'World'
                                 );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


In [13]:
%%sql
SELECT *
  FROM facts
 WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


## Observations

Not a lot of regions in the world have more water than land. Our two examples here are large territories that contain multiple distant islands as a single territory. This makes sense that our only examples are here by odd technicality; people tend to live on land, and water provides a border issue that is sometimes resolved by making the water itself the border, like the Rio Grande between the US and Mexico. Besides that, it must not be worth it to claim water as one's territory unless it is water on the coast; there's probably a whole can of worms to get into on this subject.

# Country with the greatest magnitude of population growth

Above, we stated that South Sudan had the highest population growth, but that was based on a percentage and thus could be a misleading statement. What country has the highest population growth measured in individual people? We'll use the percentage provided by the dataset that we used earlier to calculate the projected amount of people that a country will add to their population the following year.

In [14]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World'
   AND population_growth * population == (SELECT MAX(population_growth * population)
                                            FROM facts
                                           WHERE name <> 'World'
                                         );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04


## Observations

Looks like India is projected to be the biggest winner in terms of new human beings, probably beating South Sudan by a lot.

# Countries with more people dying than being born

In [15]:
%%sql
SELECT *
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY population_growth ASC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
140,po,Portugal,92090,91470,620,10825309,0.09,9.27,11.02,2.67
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24


## Observations

Amazingly, some of the countries that have a higher death rate than birth rate have a rate of population growth higher than 1. These countries are reliant on something else in order to have a population growth rate higher than 1.

This would most likely be accounted for by the `migration_rate` column; these countries have a high rate of migration, but Japan has a migration rate of 0.0, and, considering the small birth rate to death rate ratio, is unsurprisingly bleeding population numbers.

# Countries with the highest population to area ratio

In [16]:
%%sql
SELECT *, (population/area) AS population_density
  FROM facts
 WHERE (population/area) >= (SELECT AVG(population)/AVG(area)
                               FROM facts
                              WHERE name <> 'World'
                            )
 ORDER BY population_density DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_density
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299


Our results match a 2021 Google search except that Gibraltar is spot 5 instead of Gaza Strip. (Also the spelling of Macau is Macao in this search result, and the spell-check I have for Jupyter Notebook agrees with said Google search result.)